In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_in = pd.read_csv('train_in.csv')
train_out = pd.read_csv('train_out.csv')

In [3]:
class Perceptron_net:
    def __init__(self,input_dim, output_dim):
        self.weights = np.random.rand(input_dim + 1,output_dim)
        
    def forward(self,x):
        return np.column_stack((np.ones([x.shape[0]]),x)) @ self.weights

    def predict(self, x):
        return np.concatenate((np.ones(1),x)) @ self.weights 

    def fit(self,x,y,learning_rate = 1):
        # Getting the predictions for train set
        predictions = np.argmax(self.forward(x),axis = 1)
        # Getting an array where we have the indexes of wrongly predicted inputs
        wrong_examples = np.where(predictions != y.squeeze())[0].squeeze()

        # Until we have any wrong example
        while wrong_examples.size != 0:
            for i in wrong_examples: 
                #Getting a prediction for one of the first 
                predictions = self.predict(x.iloc[i])
                y_p = np.argmax(predictions)
                y_tmp  = y.iloc[i]['6']
                # comparison to the label from train_out
                if y_p != y_tmp:
                    # If it's still wrong, update the weights according to the algorithm
                    for j in range(10):
                        if predictions[j] > predictions[y_tmp]:
                            self.weights[:,j] = self.weights[:,j] - learning_rate*np.concatenate((np.ones(1),x.iloc[i]))
                        elif j == y_tmp:
                            self.weights[:,j] = self.weights[:,j] + learning_rate*np.concatenate((np.ones(1),x.iloc[i]))
            # After the for function, we check if every example is right, and create an array with indexes of wrongly predicted examples
            predictions = np.argmax(self.forward(x),axis = 1)
            wrong_examples = np.where(predictions != y.squeeze())[0]

            # This is just a technical thing, when there is only 1 wrong example, the for function cannot iterate on it,
            #  so we squeeze only if the size of the array of wrong examples is longer than 1
            if wrong_examples.size != 1:
                wrong_examples = wrong_examples.squeeze()

Creating the Model

In [9]:
model = Perceptron_net(256,10)

Fitting the model

In [10]:
%%time
model.fit(train_in, train_out)

CPU times: total: 1.06 s
Wall time: 2.11 s


Checking the accuracy of the model on the train set (we should get 100%)

In [11]:
out = np.argmax(model.forward(train_in),axis = 1)
sq = train_out.squeeze()
tmp = np.array(sq == out).astype(int)
np.sum(tmp)/tmp.shape[0]
# out == train_out[:]

1.0

Testing the model on the test set

In [12]:
test_in = pd.read_csv('test_in.csv')
test_out = pd.read_csv('test_out.csv')

In [13]:
out = np.argmax(model.forward(test_in),axis = 1)
sq = test_out.squeeze()
tmp = np.array(sq == out).astype(int)
np.sum(tmp)/tmp.shape[0]

0.8788788788788788

In [14]:
train_in.shape

(1706, 256)